In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from config import gkey
import time

In [5]:
#Import City CSV and Check file format
city_df=pd.read_csv("CSV_Data_Files/us_cities_list.csv")
city_df.head()

,city,state,city_state,country
0,New York,NY,"New York, NY",United States
1,San Francisco,CA,"San Francisco, CA",United States
2,Honolulu,HI,"Honolulu, HI",United States
3,Anchorage,AK,"Anchorage, AK",United States
4,Brooklyn,NY,"Brooklyn, NY",United States


In [7]:
#Import Cities and Keyword Search for Google API Call
cities = city_df["city_state"]
interest_type = ["hotel","amusement_park","bar","cafe","casino","museum","night_club","restaurant","shopping_mall","spa","stadium","zoo"]

#Prepare Dataframe
places_df = pd.DataFrame([])

#Run API Call
for city in cities:
    location= city
    
    for interest in interest_type:
        item = interest
        
        try:

            query=item+" in "+location

            # Build URL and Generate Call
            url="https://maps.googleapis.com/maps/api/place/textsearch/json?query="+query+"&key="+gkey
            response_json = requests.get(url).json()

            #Pull Data from Call and generate lists (**WE CAN ADD STUFF HERE**) 
            name = [item["name"] for item in response_json["results"]]
            rating = [item["rating"] for item in response_json["results"]]
            total_ratings = [item["user_ratings_total"] for item in response_json["results"]]

            #Generate Initial DataFrame
            places_df = places_df.append(pd.DataFrame({"name":name,
                                                       "rating":rating, 
                                                       "total reviews": total_ratings, 
                                                       "Interest Type": interest, 
                                                       "City": location}), ignore_index = True)

        except:
            print("not found")

        continue
    
        

not found
not found
not found
not found
not found
not found
not found
not found


In [8]:
#Build out Counts & Averages 
places_df.head()

,name,rating,total reviews,Interest Type,City
0,InterContinental New York Times Square,4.3,2120.0,hotel,"New York, NY"
1,Freehand New York,4.4,848.0,hotel,"New York, NY"
2,Cachet Boutique Hotel NYC,4.0,238.0,hotel,"New York, NY"
3,The Pearl Hotel New York,4.6,376.0,hotel,"New York, NY"
4,City Club Hotel,4.1,299.0,hotel,"New York, NY"


In [9]:
#Write Data from Places Call to CSV
places_df.to_csv("Output/city_interest_data.csv", index=False, header=True)

In [14]:
#Develop Data Frame ot the Ratings Summary by City and Interest

#Groupby City & Interest Type
places_count_df = places_df.groupby(['City', 'Interest Type'])

#Summary Calculations
count_ratings = places_count_df['rating'].count()
average_ratings = round(places_count_df['rating'].mean(),2)
max_reviews = places_count_df['total reviews'].max()
average_reviews = round(places_count_df['total reviews'].mean(),2)


In [15]:
#insert code to count ratings greater than three to add to dataframe below
#places_count_df[places_count_df['rating'] > 3].groupby('City')['Interest Type'].count()

In [16]:
#Final Data Frame
summary_rating = pd.DataFrame({'Total Rating Counts': count_ratings, 
                               'Average of Ratings': average_ratings,
                              'Max Number of Reviews': max_reviews, 
                              'Average Number of Reviews': average_reviews})

summary_rating["city_sort"] = summary_rating.index.get_level_values(0)
summary_rating["interest_sort"] = summary_rating.index.get_level_values(1)

summary_rating.head(20)

Total Rating Counts  Average of Ratings  \
City             Interest Type                                             
Albany,  NY      amusement_park                   16                4.36   
                 bar                              20                4.42   
                 cafe                             20                4.51   
                 casino                           20                3.54   
                 hotel                            20                3.94   
                 museum                           20                3.92   
                 night_club                       20                4.30   
                 restaurant                       20                4.54   
                 shopping_mall                     2                4.20   
                 spa                              20                4.15   
                 stadium                           3                4.27   
                 zoo                              20                4.44   
Albuquerque,  NM amusement_park                    1                4.40   
                 bar                              20                4.24   
                 cafe                             20                4.46   
                 casino                           15                3.63   
                 hotel                            20                4.11   
                 museum                           20                4.54   
                 night_club                       20                3.23   
                 restaurant                       20                4.52   

                                 Max Number of Reviews  \
City             Interest Type                           
Albany,  NY      amusement_park                 2333.0   
                 bar                             948.0   
                 cafe                            761.0   
                 casino                         9836.0   
                 hotel                          1259.0   
                 museum                         1455.0   
                 night_club                      948.0   
                 restaurant                      761.0   
                 shopping_mall                  4816.0   
                 spa                             129.0   
                 stadium                        2084.0   
                 zoo                            2741.0   
Albuquerque,  NM amusement_park                 2306.0   
                 bar                            1725.0   
                 cafe                           1581.0   
                 casino                         9703.0   
                 hotel                          1756.0   
                 museum                         6153.0   
                 night_club                      872.0   
                 restaurant                     4003.0   

                                 Average Number of Reviews         city_sort  \
City             Interest Type                                                 
Albany,  NY      amusement_park                     344.19       Albany,  NY   
                 bar                                398.40       Albany,  NY   
                 cafe                               209.35       Albany,  NY   
                 casino                            1968.05       Albany,  NY   
                 hotel                              457.90       Albany,  NY   
                 museum                             134.55       Albany,  NY   
                 night_club                         204.05       Albany,  NY   
                 restaurant                         382.60       Albany,  NY   
                 shopping_mall                     2930.50       Albany,  NY   
                 spa                                 27.95       Albany,  NY   
                 stadium                            776.33       Albany,  NY   
                 zoo                       

In [17]:
#Write Final Dataset to CSV
summary_rating.to_csv("Output/city_interest_rating_summary.csv", index=False, header=True)